In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from qat.lang.AQASM import AbstractGate, Program
from qat.core.console import display

In [ ]:
def Loteria(n):
    #Max number of elememnts I can generate
    N_max = 2**n
    print('N_max: {}'.format(N_max))
    #Generate winner number
    Winner = np.random.randint(N_max)
    print('Winner: {}'.format(Winner))
    return Winner#, ToBinnary(Winner, n)

In [ ]:
def TensorialHaddamard(n):
    """
    Rutina que implementa una Puerta Hadamard n tensorial
    """
    from qat.lang.AQASM import QRoutine, H
    tensorialHadamard = QRoutine()
    qbits=tensorialHadamard.new_wires(n)
    for i in range(n):
        tensorialHadamard.apply(H,qbits[i])
    return tensorialHadamard
#Creo una puerta utilizando el circuito
TensorialHaddamard_Gate = AbstractGate(
    "HaddamardTensorial", 
    [int], 
    circuit_generator=TensorialHaddamard,
    arity = lambda x: x
)

def Reflection(n, state, Positive=True):
    """
    Implementa una matriz de reflexion de dimensión n en torno a un estado dado.
    Positive:
        * True: I-2|w><w|
        * False: 2|w><w|-I
    """
    #Matriz Identidad
    Identity = np.identity(2**n)
    Identity[state, state] =-1
    if Positive:
        return Identity
    else:
        return -Identity
#Creo una puerta utilizando el circuito    
Reflexion_Gate = AbstractGate(
    "Reflexion", 
    [int, int, bool], 
    matrix_generator=Reflection,
    arity = lambda x, y, z: x
)

def Difusor(n):
    """
    Rutina que implementa el Difusor en n dimensiones
    """        
    from qat.lang.AQASM import QRoutine
    Difusor_rout = QRoutine()
    wires = Difusor_rout.new_wires(n)
    #Aplicamos n Hadamard
    Difusor_rout.apply(TensorialHaddamard_Gate(n),wires)
    Difusor_rout.apply(Reflexion_Gate(n, 0, False),wires)
    Difusor_rout.apply(TensorialHaddamard_Gate(n),wires)
    return Difusor_rout
#Creo una puerta utilizando el circuito
Difusor_Gate = AbstractGate(
    "Difusor", 
    [int], 
    circuit_generator=Difusor,
    arity = lambda x: x
)

def Grover(n, state, r):
    """
    Implementa r iteraciones Grover
    n: number of qbits
    state: number with the winner state
    r: number of applications of Grover operator
    """
    from qat.lang.AQASM import QRoutine
    Grover_rout = QRoutine()
    wires = Grover_rout.new_wires(n)
    for i in range(r):
        Grover_rout.apply(Reflexion_Gate(n, state, True),wires)
        Grover_rout.apply(Difusor_Gate(n),wires)
    return Grover_rout
#Creo una puerta utilizando el circuito
Grover_Gate = AbstractGate(
    "Grover", 
    [int, int, int ], 
    circuit_generator=Grover,
    arity = lambda x, y, z: x
)


def DoAGrover(n, Winner, r):
    """
    Implementa un algoritmo Grover con r iteraciones
    """
    if r >0:
        #Definimos el circuito
        Circuit = Program()
        #Reservamos los qbits que queremos
        qbits = Circuit.qalloc(n)
        #Generamos una superposición equiprobable de estados
        Circuit.apply(TensorialHaddamard_Gate(n),qbits)
        Circuit.apply(Grover_Gate(n, Winner, r),qbits)
        return Circuit
    else:
        raise ValueError('EL numero de iteraciones del algoritmo de Grover debe ser mayor que 0')

In [ ]:
def OptimalGrover(n):
    """
    En base al numero de qbits calcula el angulo de rotación de una iteración Grover
    y el numero optimo de iteraciones Grover para maximizar la probabilidad
    """
    #Rotation angle for Glover Algorithm
    Theta = 2*np.arcsin(np.sqrt(1/(2**n)))    
    #Number of times for application of Grover Algorithm
    r = (np.pi/(2*Theta))-0.5
    return int(np.round(r))

def Do(n, state, r):
    Circuit = Program()
    qbits = Circuit.qalloc(n)
    Circuit.apply(TensorialHaddamard_Gate(n),qbits)
    for i in range(r):
        Circuit.apply(Reflexion_Gate(n, state, True),qbits)
        Circuit.apply(Difusor_Gate(n),qbits)
        
    from qat.qpus import LinAlg
    linalgqpu = LinAlg()
    Job = Circuit.to_circ().to_job(nbshots=1)
    result = linalgqpu.submit(Job)
    Solucion=str(result[0].state)m
    SolucionDecimal = int(Solucion.replace('|','').replace('>', ''),2)
    #print('SolucionDecimal: {}'.format(SolucionDecimal))
    if SolucionDecimal == state:
        return True
    else:
        return False   

In [ ]:
def Probabilities4Groover(n, state, r):
    """
    Calcula las probabilidades de r iteraciones  Groover
    """
    Circuit = DoAGrover(n, state, r)
    from qat.qpus import PyLinalg
    linalgqpu = PyLinalg()
    Job = Circuit.to_circ().to_job()
    result = linalgqpu.submit(Job)
    States = []
    Probabilities =[]
    DecimalNumbers = []
    #Para ver como funciona calculamos todos los estados posibles
    #Y sus probabilidades asociadas

    for sample in result:
        States.append(str(sample.state).replace('|','').replace('>', ''))
        #Probabilidad del estado
        Probabilities.append(np.absolute(sample.amplitude)**2)
        DecimalNumbers.append(int(str(sample.state).replace('|','').replace('>', ''),2))
    return Probabilities, States, DecimalNumbers
    #Buscamos el Estado con la mayor probabilidad
    #idMax = Probabilities.index(max(Probabilities))
    #Ganador=str(States[idMax]).replace('|','').replace('>', '')
    #print('Estado con la mayor Probabilidad: {} -> {}. Probabilidad: {}'.format(
    #    Ganador, int(Ganador,2), Probabilities[idMax]))  
    
def GroverRoutine(n, state, r):
    """
    Obtiene una medida de r iteraciones Groover 
    """
    Circuit = DoAGrover(n, state, r)
    #Le pido una medida
    from qat.qpus import LinAlg
    linalgqpu = LinAlg()
    Job4Groover = Circuit.to_circ().to_job(nbshots=1)
    result = linalgqpu.submit(Job4Groover)
    Solucion=str(result[0].state)
    SolucionDecimal = int(Solucion.replace('|','').replace('>', ''),2)
    #print('SolucionDecimal: {}'.format(SolucionDecimal))
    if SolucionDecimal == state:
        return True
    else:
        return False       

Vamos a comprobar que todo funciona bien. Para ello fijamos un número de Qbits y vamos a hacer un barrido en el número de iteraciones Grover y registramos la probabilidad de obtener el elemento ganador. Podemos generar un gráfico Probabilidad vs iteraciones y comprobamos que el máximo está donde tiene que estar (lo calculamos llamando a la función OptimalGrover)

In [ ]:
Winner = Loteria(10)

In [ ]:
OptimalIterations = OptimalGrover(10)

In [ ]:
prob, state, numbers = Probabilities4Groover(10, Winner, OptimalIterations)

In [ ]:
prob[numbers.index(Winner)]

In [ ]:
state[numbers.index(Winner)]

In [ ]:
numbers.index(Winner)

In [ ]:
NumberOfQbits = 'll'
Probs = []
Max = []
NumberOfQbits = []
for nq in range(2, 11):
    NumberOfQbits.append(nq)
    Winner = Loteria(nq)
    OptimalIterations = OptimalGrover(nq)
    Max.append(OptimalIterations)
    print('Iteraciones óptimas: {}'.format(OptimalIterations))
    WinnerProbabilities = []
    for i in range(1, 2*OptimalIterations+1):
        prob, state, numbers = Probabilities4Groover(nq, Winner, i)
        #Obtengo la probabilidad del numero ganador
        WinnerProbabilities.append(prob[numbers.index(Winner)])
    Probs.append(WinnerProbabilities)
    
        
    
Data = pd.DataFrame(Probs, index=NumberOfQbits).T    
Data.index=range(1, len(Data)+1)

In [ ]:
import pandas as pd

In [ ]:
Data = pd.DataFrame(Probs, index=NumberOfQbits).T   
Data.index=range(1, len(Data)+1)


In [ ]:
MaximosTeoricos = pd.Series(Max, index=range(2, len(Max)+2))

In [ ]:
for columns in Data.columns:
    plt.plot(Data[columns], 'o-')
plt.xlabel('Iteraciones')    
plt.ylabel('Probabilidad de Ganar')  
plt.legend(Data.columns)

In [ ]:
#Localizacion de los másximos experimentales
Data.idxmax(axis=0)

In [ ]:
#Localizacion de los máximos teóricos
MaximosTeoricos

In [ ]:
Data.idxmax(axis=0) == MaximosTeoricos

Parece que está bien implementado

## Rutina Groover

La función **GroverRoutine** lo que hace es a partir de un número de Qbits y del elemento ganador montar un algoritmo Grover que se ejucutará r veces. A continuación hace una sola medida y la compara con la del número ganador y devuelve True o False en función de si acierta o no. 
Vamos a comparar la rutina cuántica y la clásica

In [ ]:
def GroverRoutine(n, Winner, r):
    Circuit = DoAGrover(n, Winner, r)
    #Le pido una medida
    from qat.qpus import LinAlg
    linalgqpu = LinAlg()
    Job4Groover = Circuit.to_circ().to_job(nbshots=1)
    result = linalgqpu.submit(Job4Groover)
    Solucion=str(result[0].state)
    SolucionDecimal = int(Solucion.replace('|','').replace('>', ''),2)
    #print('SolucionDecimal: {}'.format(SolucionDecimal))
    if SolucionDecimal == Winner:
        return True
    else:
        return False   

In [ ]:
max(Probabilities4Groover(8, Winner, 12)[0])

In [ ]:
int(str(GroverRoutine(8, Winner, 6)[0].state).replace('|','').replace('>', ''),2)

In [ ]:
def Oraculo(asked, winner):
    #print('Your Input is: {}'.format(asked))
    if asked == winner:
        print('You Win')
        return True
    else:
        #print('You Lose')
        return False
def GetNumber01(N, winner):
    i=0
    Continue = True
    NumberOfAsks = 0
    while Continue:
        Condition = Oraculo(i, winner)
        NumberOfAsks = NumberOfAsks +1
        i=i+1
        if Condition:
            Continue = False
        if NumberOfAsks > 2**N:
            print('Algo salio mal')
            Continue = False
    print('Number Of Questions :{}'.format(NumberOfAsks)) 
    return NumberOfAsks   

In [ ]:
def LoopGrover(n, Winner):
    i=1
    Continue = True    
    while Continue:
        print(i)
        Condition = GroverRoutine(n, Winner, i)
        print(Condition)
        if Condition:
            Continue = False
        else:
            i=i+1
        if i > 2**n:
            print('Algo salio mal')
            Continue = False
        
    #print('Number Of Questions :{}'.format(i)) 
    return i           

In [ ]:
LoopGrover(8, Winner)

In [ ]:
NumberOfQbits = 8
C=[]
for i in range(100):
    Winner = Loteria(NumberOfQbits)
    C.append(LoopGrover(NumberOfQbits, Winner))

In [ ]:
plt.hist(C)

In [ ]:
np.mean(C)

In [ ]:
OptimalGrover(8)

In [ ]:
Clasico = []
Cuantico = []
NumberOfQbits = 8

for i in range(1000):
    #Lanzamos el número
    Winner = Loteria(NumberOfQbits)
    #Ejecutamos el algoritmo clásico
    Clasico.append(GetNumber01(NumberOfQbits, Winner))
    #Ahora vamos con el Groover
    Cuantico.append(LoopGrover(NumberOfQbits, Winner))

In [ ]:
plt.hist(Clasico, bins=10)
plt.hist(Cuantico, bins=10)
plt.xlabel('Intentos')
plt.ylabel('Número Aciertos')
plt.legend(['Clasico', 'Cuantico'])

In [ ]:
pdClassic = pd.Series(Clasico).value_counts().sort_index()
pdCuantico = pd.Series(Cuantico).value_counts().sort_index()

In [ ]:
plt.plot(pdCuantico, '-o')
#plt.plot(pdClassic, '-o')

Para 8 qbits deberíamos haber encontrado que la mayor probabilidad es con 12 iteraciones pero 12 iteraciones da lugar a una probabilidad muy muy alta de acertar: 0.9999470421032387. La cosa es que con solo 6 iteraciones la probabilidad de acierto es de 0.5276176773084149 y con 7 mayor de 0.65 por lo que en una sola tirada no es necesario iterar tanto para obtener el resultado correcto la mayor parte de las veces!!!

In [ ]:
max(Probabilities4Groover(8, Winner, 7)[0])

In [ ]:
Probabilities4Groover(NumberOfQbits, Winner, 7)

In [ ]:
print('Numero de llamdas promedio Clásico: {}'.format(np.mean(Clasico)))
print('Numero de llamdas promedio Cuantico: {}'.format(np.mean(Cuantico)))

In [ ]:
plt.scatter(pdClassic.index, pdClassic)
plt.scatter(pdCuantico.index, pdCuantico)

In [ ]:
129**.5